In [1]:
# Import libraries
import numpy as np

import pandas as pd
import os
import datetime
from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pyplot

import platform
from matplotlib import font_manager, rc

import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Prepare LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from tensorflow.keras import optimizers

In [4]:
# Data preprocessing
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size+1):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size-1]))
    return np.array(feature_list), np.array(label_list)

def make_FCset(data, window_size):
    feature_list = []
    for i in range(len(data) - window_size+1):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
    return np.array(feature_list)

In [6]:
# Reading the input data
data_ori= pd.read_csv('total_data.csv.csv')

In [9]:
data_ori.index=range(len(data_ori))

In [13]:
targets=['warning']
variables=['Minimum temperature',
       'Rainfall', 'Relative huminity', 'Day length',
       'Soil temperature at 3m',
       'Soil temperature at 5m', 'last_rate']

In [16]:
u_data=data_ori

In [17]:
# model input
scaler=MinMaxScaler()
window_size=3
layer_num=30
TT_year=2017

In [21]:
# train set
data_train=data_ori[data_ori['Year']<TT_year]

In [22]:
# test set
data_test=data_ori[len(data_train)-window_size+1:]
data_test.index=range(len(data_test))

In [23]:
train = data_train[variables+targets]
test = data_test[variables+targets]

In [24]:
feature_cols = variables
label_cols = ['warning']

train_feature = train[feature_cols]
train_label = train[label_cols]

test_feature = test[feature_cols]
test_label = test[label_cols]

In [25]:
# preprocess train set
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

In [26]:
# preprocess test set
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

In [27]:
# split data into train-validation
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

In [28]:
# prepare result save
result_LSTM=pd.DataFrame(data_train[['Year','Week','warning']])
predict_LSTM=pd.DataFrame(data_test[['Year','Week','warning']])

In [29]:
result_LSTM=result_LSTM[2:]
result_LSTM.index=range(len(result_LSTM))

In [30]:
predict_LSTM=predict_LSTM[2:]
predict_LSTM.index=range(len(predict_LSTM))

In [31]:
# LSTM
num=0
fit_num=1000

for i in range(fit_num):
    model = Sequential()
    model.add(LSTM(layer_num, 
                   input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                   activation='relu', 
                   return_sequences=False)
              )
    sgd=optimizers.SGD(lr=0.001)
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', patience=50)
    filename =  'tmp_checkpoint.h5'
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

    history = model.fit(x_train, y_train, 
                        epochs=5000, 
                        batch_size=30,
                        validation_data=(x_valid, y_valid), 
                        callbacks=[early_stop, checkpoint])
    
    aa=pd.DataFrame(model.predict(train_feature),columns={'a'})
    bb=pd.DataFrame(model.predict(test_feature),columns={'a'})
    
    
    result_LSTM['Sim_'+ str(i)]=aa['a']
    predict_LSTM['Sim_'+ str(i)]=bb['a']
    

    num=num+1


Epoch 1/5000
 1/12 [=>............................] - ETA: 0s - loss: 4.4479
Epoch 00001: val_loss improved from inf to 1.74181, saving model to tmp_checkpoint.h5
12/12 [==============================] - 1s 64ms/step - loss: 2.5571 - val_loss: 1.7418
Epoch 2/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.9644
Epoch 00002: val_loss improved from 1.74181 to 1.09821, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 10ms/step - loss: 0.9922 - val_loss: 1.0982
Epoch 3/5000
 1/12 [=>............................] - ETA: 0s - loss: 1.1173
Epoch 00003: val_loss improved from 1.09821 to 0.93113, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 10ms/step - loss: 0.6630 - val_loss: 0.9311
Epoch 4/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.2966
Epoch 00004: val_loss improved from 0.93113 to 0.63277, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 10ms/step - loss: 0.4394 - v

Epoch 73/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0867
Epoch 00073: val_loss did not improve from 0.10756
12/12 [==============================] - 0s 7ms/step - loss: 0.0710 - val_loss: 0.1163
Epoch 74/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0670
Epoch 00074: val_loss did not improve from 0.10756
12/12 [==============================] - 0s 7ms/step - loss: 0.0763 - val_loss: 0.1184
Epoch 75/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0944
Epoch 00075: val_loss did not improve from 0.10756
12/12 [==============================] - 0s 7ms/step - loss: 0.0786 - val_loss: 0.1162
Epoch 76/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0687
Epoch 00076: val_loss did not improve from 0.10756
12/12 [==============================] - 0s 8ms/step - loss: 0.0710 - val_loss: 0.1101
Epoch 77/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0657
Epoch 00077: val_loss did not improve from 0.10756
12/12 [

12/12 [==============================] - 0s 9ms/step - loss: 0.0643 - val_loss: 0.0961
Epoch 110/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0326
Epoch 00110: val_loss did not improve from 0.09611
12/12 [==============================] - 0s 6ms/step - loss: 0.0615 - val_loss: 0.1032
Epoch 111/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0590
Epoch 00111: val_loss did not improve from 0.09611
12/12 [==============================] - 0s 6ms/step - loss: 0.0608 - val_loss: 0.1019
Epoch 112/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0897
Epoch 00112: val_loss improved from 0.09611 to 0.09595, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 11ms/step - loss: 0.0582 - val_loss: 0.0959
Epoch 113/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0538
Epoch 00113: val_loss did not improve from 0.09595
12/12 [==============================] - 0s 6ms/step - loss: 0.0582 - val_loss: 0.0995
Epoch

Epoch 147/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0371
Epoch 00147: val_loss did not improve from 0.09134
12/12 [==============================] - 0s 7ms/step - loss: 0.0578 - val_loss: 0.0961
Epoch 148/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0629
Epoch 00148: val_loss did not improve from 0.09134
12/12 [==============================] - 0s 7ms/step - loss: 0.0574 - val_loss: 0.0986
Epoch 149/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0413
Epoch 00149: val_loss improved from 0.09134 to 0.08915, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 11ms/step - loss: 0.0575 - val_loss: 0.0892
Epoch 150/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0819
Epoch 00150: val_loss did not improve from 0.08915
12/12 [==============================] - 0s 7ms/step - loss: 0.0600 - val_loss: 0.0920
Epoch 151/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0599
Epoch 00151: 

Epoch 221/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0765
Epoch 00221: val_loss did not improve from 0.07780
12/12 [==============================] - 0s 7ms/step - loss: 0.0542 - val_loss: 0.0854
Epoch 222/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0617
Epoch 00222: val_loss improved from 0.07780 to 0.07584, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 11ms/step - loss: 0.0473 - val_loss: 0.0758
Epoch 223/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0683
Epoch 00223: val_loss did not improve from 0.07584
12/12 [==============================] - 0s 7ms/step - loss: 0.0581 - val_loss: 0.1003
Epoch 224/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0455
Epoch 00224: val_loss did not improve from 0.07584
12/12 [==============================] - 0s 7ms/step - loss: 0.0556 - val_loss: 0.1563
Epoch 225/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.1838
Epoch 00225: 

...

 1/12 [=>............................] - ETA: 0s - loss: 0.2800
Epoch 00008: val_loss improved from 0.22329 to 0.21262, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 6ms/step - loss: 0.2989 - val_loss: 0.2126
Epoch 9/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.3276
Epoch 00009: val_loss did not improve from 0.21262
12/12 [==============================] - 0s 5ms/step - loss: 0.2804 - val_loss: 0.2181
Epoch 10/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.2186
Epoch 00010: val_loss improved from 0.21262 to 0.21132, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 7ms/step - loss: 0.2671 - val_loss: 0.2113
Epoch 11/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.2411
Epoch 00011: val_loss improved from 0.21132 to 0.20327, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 7ms/step - loss: 0.2423 - val_loss: 0.2033
Epoch 12/5000
 1/12 [=>...........

Epoch 43/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.1209
Epoch 00043: val_loss did not improve from 0.11704
12/12 [==============================] - 0s 5ms/step - loss: 0.0916 - val_loss: 0.1244
Epoch 44/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0947
Epoch 00044: val_loss did not improve from 0.11704
12/12 [==============================] - 0s 5ms/step - loss: 0.0920 - val_loss: 0.1220
Epoch 45/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0966
Epoch 00045: val_loss did not improve from 0.11704
12/12 [==============================] - 0s 5ms/step - loss: 0.0902 - val_loss: 0.1235
Epoch 46/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0687
Epoch 00046: val_loss did not improve from 0.11704
12/12 [==============================] - 0s 4ms/step - loss: 0.0904 - val_loss: 0.1216
Epoch 47/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0759
Epoch 00047: val_loss did not improve from 0.11704
12/12 [

Epoch 80/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0612
Epoch 00080: val_loss did not improve from 0.10871
12/12 [==============================] - 0s 5ms/step - loss: 0.0715 - val_loss: 0.1100
Epoch 81/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0591
Epoch 00081: val_loss improved from 0.10871 to 0.10841, saving model to tmp_checkpoint.h5
12/12 [==============================] - 0s 6ms/step - loss: 0.0726 - val_loss: 0.1084
Epoch 82/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0775
Epoch 00082: val_loss did not improve from 0.10841
12/12 [==============================] - 0s 5ms/step - loss: 0.0704 - val_loss: 0.1097
Epoch 83/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.1070
Epoch 00083: val_loss did not improve from 0.10841
12/12 [==============================] - 0s 4ms/step - loss: 0.0733 - val_loss: 0.1094
Epoch 84/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0863
Epoch 00084: val_lo

Epoch 117/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.1035
Epoch 00117: val_loss did not improve from 0.10297
12/12 [==============================] - 0s 6ms/step - loss: 0.0659 - val_loss: 0.1070
Epoch 118/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0545
Epoch 00118: val_loss did not improve from 0.10297
12/12 [==============================] - 0s 5ms/step - loss: 0.0694 - val_loss: 0.1160
Epoch 119/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0530
Epoch 00119: val_loss did not improve from 0.10297
12/12 [==============================] - 0s 5ms/step - loss: 0.0761 - val_loss: 0.1079
Epoch 120/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0490
Epoch 00120: val_loss did not improve from 0.10297
12/12 [==============================] - 0s 5ms/step - loss: 0.0678 - val_loss: 0.1059
Epoch 121/5000
 1/12 [=>............................] - ETA: 0s - loss: 0.0666
Epoch 00121: val_loss did not improve from 0.10297
12

KeyboardInterrupt: 

In [32]:
result_LSTM

,Year,Week,warning,Sim_0,Sim_1,Sim_2,Sim_3,Sim_4,Sim_5,Sim_6,...,Sim_52,Sim_53,Sim_54,Sim_55,Sim_56,Sim_57,Sim_58,Sim_59,Sim_60,Sim_61
0,2009,3,1,0.953216,0.992823,1.103159,0.981522,0.954820,1.032380,0.985588,...,1.030024,0.970932,0.995796,0.847411,0.964723,0.980632,0.946549,0.958823,0.886166,0.959488
1,2009,4,1,0.993442,0.962419,1.038650,1.019132,0.855650,1.011679,0.914581,...,0.971153,0.937877,0.973310,0.851773,0.920767,0.972520,0.975925,1.023174,0.916374,0.971700
2,2009,5,1,1.000077,1.031898,0.999706,0.951907,0.864714,1.044413,1.004400,...,1.030682,0.937165,0.963412,0.886624,0.956208,0.986487,1.005682,0.976297,0.926748,0.979472
3,2009,6,1,1.006787,0.942422,0.969392,1.079350,0.827706,1.013074,0.962280,...,1.116520,0.924019,1.035484,0.941622,1.015792,1.023487,1.135680,0.950921,0.933856,0.967247
4,2009,7,1,0.943427,0.965894,0.933683,1.022502,0.983076,1.046844,0.997619,...,1.092562,0.964170,1.022741,0.924228,1.057933,1.012698,1.033306,0.950047,0.849564,0.893457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2016,49,1,1.126896,1.084510,1.219404,1.040468,1.061538,1.085150,0.965244,...,1.116459,0.976446,1.068426,0.894228,1.038409,1.052869,1.171120,1.009561,0.916189,1.048458
418,2016,50,1,1.038991,1.060502,1.065889,1.060808,0.952128,1.012196,0.951040,...,1.086692,0.893240,1.015627,0.917132,1.022988,1.025177,1.114463,1.021382,0.899574,1.023762
419,2016,51,1,1.016781,1.064719,1.093541,1.065882,0.934370,0.993960,0.967442,...,1.070025,0.907818,0.994308,0.906675,1.029370,1.024292,1.074264,1.017842,0.918652,1.004836
420,2016,52,1,1.014788,0.958822,1.169607,1.044207,0.869035,1.044381,0.992160,...,0.963866,1.018608,1.108415,1.033514,1.078183,1.112117,1.161334,1.186025,0.969404,1.049034


In [33]:
predict_LSTM

,Year,Week,warning,Sim_0,Sim_1,Sim_2,Sim_3,Sim_4,Sim_5,Sim_6,...,Sim_52,Sim_53,Sim_54,Sim_55,Sim_56,Sim_57,Sim_58,Sim_59,Sim_60,Sim_61
0,2017,1,1,1.080103,1.041402,1.066062,0.998880,1.025412,0.977501,1.032085,...,1.032538,1.088293,1.236369,0.911524,1.106313,1.032872,1.218061,1.033742,0.920515,1.042659
1,2017,2,1,1.147799,1.173631,1.195086,1.076904,1.234605,0.919255,0.981518,...,1.220491,1.080442,1.108959,0.914991,1.221414,1.012376,0.951548,1.041770,0.843777,0.959113
2,2017,3,1,1.031170,1.085605,1.185383,1.001768,1.097232,1.064328,1.024561,...,1.088537,0.996073,1.074342,0.856803,1.066105,0.972617,1.000283,0.968090,0.889872,0.996724
3,2017,4,1,1.018070,1.031878,1.163282,1.033757,0.998335,1.042400,0.924437,...,0.984732,0.946365,1.099210,0.809510,0.955932,0.974121,0.998160,0.980853,0.856470,0.968148
4,2017,5,1,1.035901,0.963855,1.055065,1.066426,0.981330,1.069174,0.970480,...,1.006514,0.980549,0.950566,0.916760,0.948607,0.972163,0.999906,1.043403,0.916658,0.967806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2018,49,1,1.033239,0.974621,0.884526,0.936551,0.819499,1.014393,0.918015,...,0.962984,0.871568,0.981880,0.930677,1.009886,1.018735,1.091804,1.067184,0.871981,1.030103
102,2018,50,1,0.854326,1.021504,0.746529,0.983607,0.781722,1.059761,0.883671,...,0.929572,0.877573,0.744512,0.824913,0.882192,0.990260,0.992710,0.904360,0.786812,1.044262
103,2018,51,1,0.934204,1.025154,0.940988,0.983523,0.698648,0.860938,0.910855,...,0.903527,0.857197,0.912689,0.845132,0.914561,0.981921,1.130363,1.035527,0.924231,1.016428
104,2018,52,1,0.948096,1.102167,0.991974,1.053337,0.986948,1.099548,0.963895,...,1.001725,1.095040,0.920823,0.939483,1.112635,1.173542,0.948035,0.964867,0.843753,0.939100


In [34]:
# save the results
result_sum=result_LSTM.iloc[:,result_LSTM.columns.str.contains("Sim")]
predict_sum=predict_LSTM.iloc[:,predict_LSTM.columns.str.contains("Sim")]

In [35]:
result_sum.to_excel('result_sum_LSTM.xlsx',index=None)
predict_sum.to_excel('predict_sum_LSTM.xlsx',index=None)

In [35]:
result_sum['warning']=train_label
predict_sum['warning']=test_label

<ipython-input-35-a53f1a0d1a56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_sum['warning']=train_label
<ipython-input-35-a53f1a0d1a56>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_sum['warning']=test_label


In [36]:
result_sum

,Sim_0,Sim_1,Sim_2,Sim_3,Sim_4,Sim_5,Sim_6,Sim_7,Sim_8,Sim_9,...,Sim_53,Sim_54,Sim_55,Sim_56,Sim_57,Sim_58,Sim_59,Sim_60,Sim_61,warning
0,0.953216,0.992823,1.103159,0.981522,0.954820,1.032380,0.985588,0.932565,0.972624,0.999058,...,0.970932,0.995796,0.847411,0.964723,0.980632,0.946549,0.958823,0.886166,0.959488,1
1,0.993442,0.962419,1.038650,1.019132,0.855650,1.011679,0.914581,0.901680,1.015905,1.045723,...,0.937877,0.973310,0.851773,0.920767,0.972520,0.975925,1.023174,0.916374,0.971700,1
2,1.000077,1.031898,0.999706,0.951907,0.864714,1.044413,1.004400,0.983242,1.041623,1.011402,...,0.937165,0.963412,0.886624,0.956208,0.986487,1.005682,0.976297,0.926748,0.979472,1
3,1.006787,0.942422,0.969392,1.079350,0.827706,1.013074,0.962280,0.998096,0.949294,1.012137,...,0.924019,1.035484,0.941622,1.015792,1.023487,1.135680,0.950921,0.933856,0.967247,1
4,0.943427,0.965894,0.933683,1.022502,0.983076,1.046844,0.997619,1.038365,1.061539,1.028453,...,0.964170,1.022741,0.924228,1.057933,1.012698,1.033306,0.950047,0.849564,0.893457,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,1.126896,1.084510,1.219404,1.040468,1.061538,1.085150,0.965244,1.031429,1.213888,1.085784,...,0.976446,1.068426,0.894228,1.038409,1.052869,1.171120,1.009561,0.916189,1.048458,1
418,1.038991,1.060502,1.065889,1.060808,0.952128,1.012196,0.951040,0.992791,1.078676,1.056153,...,0.893240,1.015627,0.917132,1.022988,1.025177,1.114463,1.021382,0.899574,1.023762,1
419,1.016781,1.064719,1.093541,1.065882,0.934370,0.993960,0.967442,1.001983,1.052569,1.063257,...,0.907818,0.994308,0.906675,1.029370,1.024292,1.074264,1.017842,0.918652,1.004836,1
420,1.014788,0.958822,1.169607,1.044207,0.869035,1.044381,0.992160,1.163687,1.219319,1.050429,...,1.018608,1.108415,1.033514,1.078183,1.112117,1.161334,1.186025,0.969404,1.049034,1


In [37]:
# train result 
r_tmp=result_LSTM.iloc[:,result_LSTM.columns.str.contains("Sim")]
result_sum['LSTM_mean']=r_tmp.mean(axis=1)
result_sum['LSTM_95+']=r_tmp.quantile(axis=1,q=0.975)
result_sum['LSTM_95-']=r_tmp.quantile(axis=1,q=0.025)

<ipython-input-37-4459ead8a6b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_sum['LSTM_mean']=r_tmp.mean(axis=1)


In [38]:
# test result
p_tmp=predict_LSTM.iloc[:,predict_LSTM.columns.str.contains("Sim")]
predict_sum['LSTM_mean']=p_tmp.mean(axis=1)
predict_sum['LSTM_95+']=p_tmp.quantile(axis=1,q=0.975)
predict_sum['LSTM_95-']=p_tmp.quantile(axis=1,q=0.025)

<ipython-input-38-58e6c189687c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_sum['LSTM_mean']=p_tmp.mean(axis=1)


In [124]:
p_v=0.5

In [125]:
for i in result_sum.index:
    if (result_sum.loc[i,'LSTM_95+']>=p_v):
        result_sum.loc[i,'prediction_95+']=1
    else:
        result_sum.loc[i,'prediction_95+']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'LSTM_95+']>=p_v):
        predict_sum.loc[i,'prediction_95+']=1
    else:
        predict_sum.loc[i,'prediction_95+']=0

In [126]:
for i in result_sum.index:
    if (result_sum.loc[i,'LSTM_95-']>=p_v):
        result_sum.loc[i,'prediction_95-']=1
    else:
        result_sum.loc[i,'prediction_95-']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'LSTM_95-']>=p_v):
        predict_sum.loc[i,'prediction_95-']=1
    else:
        predict_sum.loc[i,'prediction_95-']=0

In [127]:
# accuracy scores
for i in result_sum.index:
    if (result_sum.loc[i,'LSTM_mean']>=p_v):
        result_sum.loc[i,'prediction']=1
    else:
        result_sum.loc[i,'prediction']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'LSTM_mean']>=p_v):
        predict_sum.loc[i,'prediction']=1
    else:
        predict_sum.loc[i,'prediction']=0

Result_ACC_train=pd.DataFrame(columns=['accuracy','Recall','Precision','F1'])
Result_ACC_test=pd.DataFrame(columns=['accuracy','Recall','Precision','F1'])

result_sum.loc[(result_sum['warning']==result_sum['prediction']),'equality']=1
predict_sum.loc[(predict_sum['warning']==predict_sum['prediction']),'equality']=1

result_sum=result_sum.fillna(0)
predict_sum=predict_sum.fillna(0)

Result_ACC_train.loc[0,'accuracy']=result_sum['equality'].mean()
Result_ACC_test.loc[0,'accuracy']=predict_sum['equality'].mean()

Result_ACC_train.loc[0,'Recall']=len(result_sum[(result_sum['warning']==1) & (result_sum['prediction']==1)])/len(result_sum[(result_sum['warning']==1)])
Result_ACC_train.loc[0,'Precision']=len(result_sum[(result_sum['warning']==1) & (result_sum['prediction']==1)])/len(result_sum[(result_sum['prediction']==1)])
Result_ACC_train.loc[0,'F1']=2*Result_ACC_train.loc[0,'Recall']*Result_ACC_train.loc[0,'Precision']/(Result_ACC_train.loc[0,'Recall']+Result_ACC_train.loc[0,'Precision'])

Result_ACC_test.loc[0,'Recall']=len(predict_sum[(predict_sum['warning']==1) & (predict_sum['prediction']==1)])/len(predict_sum[(predict_sum['warning']==1)])
Result_ACC_test.loc[0,'Precision']=len(predict_sum[(predict_sum['warning']==1) & (predict_sum['prediction']==1)])/len(predict_sum[(predict_sum['prediction']==1)])
Result_ACC_test.loc[0,'F1']=2*Result_ACC_test.loc[0,'Recall']*Result_ACC_test.loc[0,'Precision']/(Result_ACC_test.loc[0,'Recall']+Result_ACC_test.loc[0,'Precision'])

display(Result_ACC_train)

display(Result_ACC_test)

,accuracy,Recall,Precision,F1
0,0.971564,0.953782,0.961864,0.957806


,accuracy,Recall,Precision,F1
0,0.924528,0.866667,0.970149,0.915493
